# Number guessing game take 4
Not sure why previous version couldn't figure it out.

The idea is this:

Consider the situation: the hidden answer sequence is [1,0,2]. The player has correctly guessed the first 1, and is now trying to guess the second number, which is 0. Suppose the player is considering guessing the number 1 (wrongly). There is an entry in the q-table for this exact situation: with adequate training, guessing 1 in this case should have a weight of zero, while guessing 0 or 2 should have equal weights.

This seems airtight to me, so not sure what's going wrong.

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

from number_guess_environment_2 import NumberGuess

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
env = NumberGuess()

In [46]:
state_idx = env.observation_space.sample()
print("Random valid game state index:", state_idx)
print("Which corresponds to the state:",env.observation_states[state_idx])

random_guess_idx = env.action_space.sample()
print("\nRandom valid guess:", random_guess_idx)

print("\nNumber of entries in observation space times action space,\n"
      "in other words, the size of the observation states",env.observation_space.n*env.action_space.n )

print("\nAnd here are all those observation states:\n", env.observation_states)

Random valid game state index: 22
Which corresponds to the state: [ 1 -1 -1  1  1  0]

Random valid guess: 2

Number of entries in observation space times action space,
in other words, the size of the observation states 192

And here are all those observation states:
 [[-1 -1 -1  0  0  0]
 [-1 -1 -1  0  0  1]
 [-1 -1 -1  0  1  0]
 [-1 -1 -1  0  1  1]
 [-1 -1 -1  1  0  0]
 [-1 -1 -1  1  0  1]
 [-1 -1 -1  1  1  0]
 [-1 -1 -1  1  1  1]
 [ 0 -1 -1  0  0  0]
 [ 0 -1 -1  0  0  1]
 [ 0 -1 -1  0  1  0]
 [ 0 -1 -1  0  1  1]
 [ 0 -1 -1  1  0  0]
 [ 0 -1 -1  1  0  1]
 [ 0 -1 -1  1  1  0]
 [ 0 -1 -1  1  1  1]
 [ 1 -1 -1  0  0  0]
 [ 1 -1 -1  0  0  1]
 [ 1 -1 -1  0  1  0]
 [ 1 -1 -1  0  1  1]
 [ 1 -1 -1  1  0  0]
 [ 1 -1 -1  1  0  1]
 [ 1 -1 -1  1  1  0]
 [ 1 -1 -1  1  1  1]
 [ 2 -1 -1  0  0  0]
 [ 2 -1 -1  0  0  1]
 [ 2 -1 -1  0  1  0]
 [ 2 -1 -1  0  1  1]
 [ 2 -1 -1  1  0  0]
 [ 2 -1 -1  1  0  1]
 [ 2 -1 -1  1  1  0]
 [ 2 -1 -1  1  1  1]
 [ 0  1 -1  0  0  0]
 [ 0  1 -1  0  0  1]
 [ 0  1 -1  0  1 

In [85]:
r = np.array([-1, 1, -1, 0, 1, 1])
print("A given state in the observation states:\n",r)

print("\nHere, the first hidden number, which is a 1, has been guessed.\n"
      "Now, for the second number, 1 and 2 have both been guessed.")

print("\nLocate this state (its index/row) in the whole set of observation states:")
r = env.reorder_array(r)
state = env.get_state_from_array(r)
print(state)

print("And make sure we get the same state back the other direction:")
print(env.get_array_from_state(state))
print("where the first triplet may have been reordered to have all of the -1's in the back.")

print("\nNow just do a closure check:")
for i in range(env.observation_states.shape[0]):
    print(i)
    print(env.get_array_from_state(i))
    array = env.reorder_array(env.get_array_from_state(i))
    assert i == env.get_state_from_array(array)

A given state in the observation states:
 [-1  1 -1  0  1  1]

Here, the first hidden number, which is a 1, has been guessed.
Now, for the second number, 1 and 2 have both been guessed.

Locate this state (its index/row) in the whole set of observation states:
19
And make sure we get the same state back the other direction:
[ 1 -1 -1  0  1  1]
where the first triplet may have been reordered to have all of the -1's in the back.

Now just do a closure check:
0
[-1 -1 -1  0  0  0]
1
[-1 -1 -1  0  0  1]
2
[-1 -1 -1  0  1  0]
3
[-1 -1 -1  0  1  1]
4
[-1 -1 -1  1  0  0]
5
[-1 -1 -1  1  0  1]
6
[-1 -1 -1  1  1  0]
7
[-1 -1 -1  1  1  1]
8
[ 0 -1 -1  0  0  0]
9
[ 0 -1 -1  0  0  1]
10
[ 0 -1 -1  0  1  0]
11
[ 0 -1 -1  0  1  1]
12
[ 0 -1 -1  1  0  0]
13
[ 0 -1 -1  1  0  1]
14
[ 0 -1 -1  1  1  0]
15
[ 0 -1 -1  1  1  1]
16
[ 1 -1 -1  0  0  0]
17
[ 1 -1 -1  0  0  1]
18
[ 1 -1 -1  0  1  0]
19
[ 1 -1 -1  0  1  1]
20
[ 1 -1 -1  1  0  0]
21
[ 1 -1 -1  1  0  1]
22
[ 1 -1 -1  1  1  0]
23
[ 1 -1 -1  1  1  

In [96]:
print("\nInteractive Game")
env.be_verbose = True
env.reset()
print("\nCurrent state: (", env.state,")", env.get_dict_from_state())

while not env.is_done():
    print("\n")
    print(env.get_dict_from_state(env.step(int(input("Enter your guess 0, 1, or 2: ")))[0]),"\n")


Interactive Game
NumberGuess::reset: hidden answer list: [0, 1, 2]

Current state: ( 0 ) {'answer0': -1, 'answer1': -1, 'answer2': -1, 'guess0': 0, 'guess1': 0, 'guess2': 0, 'current_position': 0}


Enter your guess 0, 1, or 2: 0
current position: 0/2 | guess:0 | correct answer:0
{'answer0': 0, 'answer1': -1, 'answer2': -1, 'guess0': 0, 'guess1': 0, 'guess2': 0, 'current_position': 1} 



Enter your guess 0, 1, or 2: 2
current position: 1/2 | guess:2 | correct answer:1
{'answer0': 0, 'answer1': -1, 'answer2': -1, 'guess0': 0, 'guess1': 0, 'guess2': 1, 'current_position': 1} 



Enter your guess 0, 1, or 2: 0
current position: 1/2 | guess:0 | correct answer:1
{'answer0': 0, 'answer1': -1, 'answer2': -1, 'guess0': 1, 'guess1': 0, 'guess2': 1, 'current_position': 1} 



Enter your guess 0, 1, or 2: 1
current position: 1/2 | guess:1 | correct answer:1
{'answer0': 0, 'answer1': 1, 'answer2': -1, 'guess0': 0, 'guess1': 0, 'guess2': 0, 'current_position': 2} 



Enter your guess 0, 1, or 2: 